### Loading Libraries...

In [36]:
import re
import os
import bs4
import bert
import nltk
import spacy
import string
import keras
from empath import Empath
import en_core_web_trf
import pandas as pd
import numpy as np
from sklearn import *
from keras.layers import *
import collections
import tf_slim as slim
import transformers
import tensorflow as tf
import tensorflow_hub as hub
from keras.initializers import Constant
from datetime import datetime
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report, confusion_matrix
from transformers import BertTokenizer, BertModel
from bert import optimization
from bert import tokenization


import xgboost
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

### Loading Data...

In [4]:
# Loading the dataset
arguments_training = pd.read_csv('arguments-training.tsv', sep='\t')
labels_training = pd.read_csv('labels-training.tsv', sep='\t')
level1_labels_training = pd.read_csv('level1-labels-training.tsv', sep='\t')

# load value-categories.json to json object
import json
with open('value-categories.json') as f:
    value_categories = json.load(f)

In [5]:
arguments_training.head()

,Argument ID,Conclusion,Stance,Premise
0,A01001,Entrapment should be legalized,in favor of,if entrapment can serve to more easily capture...
1,A01002,We should ban human cloning,in favor of,we should ban human cloning as it will only ca...
2,A01003,We should abandon marriage,against,marriage is the ultimate commitment to someone...
3,A01004,We should ban naturopathy,against,it provides a useful income for some people
4,A01005,We should ban fast food,in favor of,fast food should be banned because it is reall...


In [6]:
labels_training.head()

,Argument ID,Self-direction: thought,Self-direction: action,Stimulation,Hedonism,Achievement,Power: dominance,Power: resources,Face,Security: personal,...,Tradition,Conformity: rules,Conformity: interpersonal,Humility,Benevolence: caring,Benevolence: dependability,Universalism: concern,Universalism: nature,Universalism: tolerance,Universalism: objectivity
0,A01001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A01002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,A01003,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,A01004,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,A01005,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# print structure of value-categories.json
print(json.dumps(value_categories, indent=4)[:400])

{
    "Self-direction: thought": {
        "Be creative": [
            "allowing for more creativity or imagination",
            "being more creative",
            "fostering creativity",
            "promoting imagination"
        ],
        "Be curious": [
            "being the more interesting option",
            "fostering curiosity",
            "making people more keen to learn",
       


## Cleaning Data...

In [8]:
def roomba(item):
    '''
    input: string
    output: list of cleaned words

    '''
    soup = bs4.BeautifulSoup(item, 'html.parser')
    text = soup.get_text()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english')

    text = re.sub(r'\[.*\]\(.*\)', '', text)

    # remobe '[removed]' and '[deleted]'
    text = re.sub(r'\[.*\]', '', text)

    # remove non utf-8 characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    # lowercase 
    text = text.lower()

    # remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # remove numbers
    text = re.sub(r'\d+', '', text)

    # remove extra spaces
    text = re.sub(r'\s+', ' ', text)

    # remove leading and trailing spaces
    text = text.strip()

    # remove urls
    text = re.sub(r'http\S+', '', text)

    # remove stopwords
    text = [word for word in text.split() if word not in stopwords]
    text = [lemmatizer.lemmatize(word) for word in text]


    if text == []:
        return False
    else:
        return text

In [9]:
import nltk

# create a tokenizer with NLTK
from nltk.tokenize import word_tokenize
tokenizer = word_tokenize

# create a lemmatizer with NLTK
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [10]:
# run the roomba function on the arguments_training dataframe append a 'cleaned' column
arguments_training['Cleaned_Premise'] = arguments_training['Premise'].apply(roomba)
arguments_training['Cleaned_Conclusion'] = arguments_training['Conclusion'].apply(roomba)

# combine 'conclusion', 'premise', and 'stance' into one column, then run roomba function
arguments_training['Cleaned_BOW'] = arguments_training['Premise'] + ' ' + arguments_training['Conclusion'] + ' ' + arguments_training['Stance']

c:\Users\sucho\anaconda3\envs\py39\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [11]:
arguments_training.head()

,Argument ID,Conclusion,Stance,Premise,Cleaned_Premise,Cleaned_Conclusion,Cleaned_BOW
0,A01001,Entrapment should be legalized,in favor of,if entrapment can serve to more easily capture...,"[entrapment, serve, easily, capture, wanted, c...","[entrapment, legalized]",if entrapment can serve to more easily capture...
1,A01002,We should ban human cloning,in favor of,we should ban human cloning as it will only ca...,"[ban, human, cloning, cause, huge, issue, bunc...","[ban, human, cloning]",we should ban human cloning as it will only ca...
2,A01003,We should abandon marriage,against,marriage is the ultimate commitment to someone...,"[marriage, ultimate, commitment, someone, peop...","[abandon, marriage]",marriage is the ultimate commitment to someone...
3,A01004,We should ban naturopathy,against,it provides a useful income for some people,"[provides, useful, income, people]","[ban, naturopathy]",it provides a useful income for some people We...
4,A01005,We should ban fast food,in favor of,fast food should be banned because it is reall...,"[fast, food, banned, really, bad, health, costly]","[ban, fast, food]",fast food should be banned because it is reall...


In [12]:
# merge labels-training and arguments-training on ID
training_data = pd.merge(labels_training, arguments_training, on='Argument ID')

In [13]:
training_data.columns

Index(['Argument ID', 'Self-direction: thought', 'Self-direction: action',
       'Stimulation', 'Hedonism', 'Achievement', 'Power: dominance',
       'Power: resources', 'Face', 'Security: personal', 'Security: societal',
       'Tradition', 'Conformity: rules', 'Conformity: interpersonal',
       'Humility', 'Benevolence: caring', 'Benevolence: dependability',
       'Universalism: concern', 'Universalism: nature',
       'Universalism: tolerance', 'Universalism: objectivity', 'Conclusion',
       'Stance', 'Premise', 'Cleaned_Premise', 'Cleaned_Conclusion',
       'Cleaned_BOW'],
      dtype='object')

## Word embeddings

In [14]:
# import embedding models
path_to_gloVe_file = "glove.6B.200d.txt"

# dictionary of words to embeddings
embeddings_index = {}

with open(path_to_gloVe_file, encoding='utf8') as f:
  for line in f:
    word, coefs = line.split(maxsplit = 1)
    coefs = np.fromstring(coefs, "f", sep = " ")
    embeddings_index[word] = coefs

In [15]:
# find argument with longest length of words
max_tokens = training_data['Cleaned_BOW'].str.split().str.len().max()
data = training_data['Cleaned_BOW']

# This text vectorizer indexs our vocabulary based on the train samples
vectorizer = TextVectorization(max_tokens = max_tokens, output_sequence_length = 100, split = 'whitespace')
data_tensor = tf.data.Dataset.from_tensor_slices(data).batch(32)
vectorizer.adapt(data_tensor)

# map unique words to integers
vocabulary = vectorizer.get_vocabulary()
index = dict(zip(vocabulary, range(len(vocabulary))))
len_vocabulary = len(vocabulary)

In [16]:
# create embedding matrix
embedding_dim = 200

# create a counter of words in the training set also in the embedding set
hits = 0
misses = 0

# prepare embedding matrix for embedding layer (ref. code tutorial 7)
embedding_matrix = np.zeros((len_vocabulary, embedding_dim))
for word, i in index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector
    hits+=1
  else:
    misses+=1

# create embedding layer for our network
embedding_layer = Embedding(len_vocabulary, embedding_dim, embeddings_initializer=Constant(embedding_matrix), trainable = False)

# print ratio of words in embedding set to words in training set
print("Embedding coverage: ", round((hits/(hits+misses))*100,2),"%")
print("Captured words: ", hits)
print("Missed words: ", misses)

Embedding coverage:  97.92 %
Captured words:  141
Missed words:  3


## Value Categories

In [38]:
value_category = list(value_categories.keys())
# category_value = list(value_categories.values())

In [39]:
value_category

['Self-direction: thought',
 'Self-direction: action',
 'Stimulation',
 'Hedonism',
 'Achievement',
 'Power: dominance',
 'Power: resources',
 'Face',
 'Security: personal',
 'Security: societal',
 'Tradition',
 'Conformity: rules',
 'Conformity: interpersonal',
 'Humility',
 'Benevolence: caring',
 'Benevolence: dependability',
 'Universalism: concern',
 'Universalism: nature',
 'Universalism: tolerance',
 'Universalism: objectivity']

In [17]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

# create an empty dictionary to store the tokens
tokens = {}

# iterate over the human value categories in the data
for category, values in value_categories.items():
  # create an empty list to store the tokens for the category
  tokens[category] = []
  # iterate over the examples of expressions for the category
  for examples in values.values():
    # flatten the list of examples into a single-level list of tokens
    tokens[category].extend([token for example in examples for token in example.split()])

# print the first 10 tokens for the category 'Self-direction: thought'
print(tokens['Self-direction: thought'][:10])

# pre-process tokens
for category in tokens.keys():
  # convert to lowercase
  tokens[category] = [token.lower() for token in tokens[category]]
  # remove stop words
  tokens[category] = [token for token in tokens[category] if token not in stop_words]
  # lemmatize
  tokens[category] = [lemmatizer.lemmatize(token) for token in tokens[category]]

# print the first 10 processed tokens for the category 'Self-direction: thought'
print(tokens['Self-direction: thought'][:10])

['allowing', 'for', 'more', 'creativity', 'or', 'imagination', 'being', 'more', 'creative', 'fostering']
['allowing', 'creativity', 'imagination', 'creative', 'fostering', 'creativity', 'promoting', 'imagination', 'interesting', 'option']


## Feature Extraction

In [19]:
# create a list of all value categories and subcategories
all_categories = []
for category, subcategories in value_categories.items():
    all_categories.append(category)
    for subcategory, _ in subcategories.items():
        all_categories.append(subcategory)

In [21]:
# load the arguments and labels data from the tsv files
arguments_data = {}
with open('arguments-training.tsv') as f:
    for line in f:
        argument_id, conclusion, stance, premise = line.strip().split('\t')
        arguments_data[argument_id] = {
            'premise': premise,
            'conclusion': conclusion,
            'stance': stance,
            'categories': []
        }

with open('labels-training.tsv') as f:
    for line in f:
        argument_id, *labels = line.strip().split('\t')
        arguments_data[argument_id]['categories'] = [
            category
            for category, label in zip(all_categories, labels)
            if label == '1'
        ]

In [22]:
# create a list of all arguments and their corresponding categories and subcategories
arguments = []
categories = []
for argument_id, argument_data in arguments_data.items():
    premise = argument_data['premise']
    conclusion = argument_data['conclusion']
    stance = argument_data['stance']

    # create the argument text by concatenating the premise and conclusion
    argument_text = premise + ' ' + conclusion

    # create a binary feature for each value category and subcategory
    argument_categories = []
    for category in all_categories:
        if category in argument_data['categories']:
            argument_categories.append(1)
        else:
            argument_categories.append(0)

    # add the argument text and categories to the list
    arguments.append(argument_text)
    categories.append(argument_categories)

# create a CountVectorizer to convert the argument text to numeric features
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(arguments)

# combine the numeric features with the binary category features
X_with_categories = np.hstack((X.todense(), np.array(categories)))

In [23]:
value_category_list = list(value_categories.keys())

# create empty list to store feature vectors
feature_vectors = []

# iterate over arguments
for argument in arguments_training:
    argument_id = argument[0]
    conclusion = argument[1]
    stance = argument[2]
    premise = argument[3]

    # create empty feature vector
    feature_vector = []

    # add conclusion, stance, and premise to feature vector
    feature_vector.append(conclusion)
    feature_vector.append(stance)
    feature_vector.append(premise)

    # add length of conclusion, stance, and premise to feature vector
    feature_vector.append(len(conclusion))
    feature_vector.append(len(stance))
    feature_vector.append(len(premise))

    # add number of words in conclusion, stance, and premise to feature vector
    feature_vector.append(len(conclusion.split()))
    feature_vector.append(len(stance.split()))
    feature_vector.append(len(premise.split()))

    # add value category information to feature vector
    for value_category in value_category_list:
        if argument_id in labels_training and value_category in labels_training[argument_id]:
            feature_vector.append(1)
        else:
            feature_vector.append(0)

        # append feature vector to list of feature vectors
        feature_vectors.append(feature_vector)

# print first 5 feature vectors
print(feature_vectors[:5])

[['r', 'g', 'u', 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], ['r', 'g', 'u', 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], ['r', 'g', 'u', 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], ['r', 'g', 'u', 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], ['r', 'g', 'u', 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [37]:
# Import necessary libraries
import json
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

# Load the value categories from the JSON file
with open('value-categories.json') as f:
  value_categories = json.load(f)

# Create a dictionary of words and phrases for each value category
value_dict = {}
for category, subcategories in value_categories.items():
  words = []
  for subcategory, definitions in subcategories.items():
    for definition in definitions:
      words += definition.split()
  value_dict[category] = set(words)

# Load the training data
arguments = pd.read_csv('arguments-training.tsv', sep='\t')
labels = pd.read_csv('labels-training.tsv', sep='\t')

# Preprocess the text data
tfidf = TfidfVectorizer(stop_words='english')
arguments_tfidf = tfidf.fit_transform(arguments['Premise'])

# Create a binary indicator for each value category
mlb = MultiLabelBinarizer()
labels_bin = mlb.fit_transform(labels.iloc[:,1:])

# Identify the values being expressed in the arguments
values = []
for i in range(arguments.shape[0]):
  argument = arguments_tfidf[i]
  values.append([])
  for category, word_set in value_dict.items():
    for word in word_set:
      if word in tfidf.vocabulary_:
        word_idx = tfidf.vocabulary_[word]
        if argument[0, word_idx] > 0:
          values[-1].append(category)
          break

# Create a feature vector representing the values in each argument
features = mlb.transform(values)

# Train a classifier to classify the arguments according to their values
classifier = OneVsRestClassifier(SVC(gamma='auto'))
classifier.fit(features, labels_bin)

# Evaluate the model on the test set
arguments_test = pd.read_csv('arguments-test.tsv', sep='\t')
arguments_tfidf_test = tfidf.transform(arguments_test['Premise'])
labels_test = pd.read_csv('labels-test.tsv', sep='\t')
labels_bin_test = mlb.transform(labels_test.iloc[:,1:])
predictions = classifier.predict(arguments_tfidf_test)

# Calculate evaluation metrics
precision = precision_score(labels_bin_test, predictions, average='micro')
recall = recall_score(labels_bin_test, predictions, average='micro')
f1 = f1_score(labels_bin_test, predictions, average='micro')

# Print evaluation metrics
print('Precision: ', precision)
print('Recall: ', recall)
print('F1-score: ', f1)

c:\Users\sucho\anaconda3\envs\py39\lib\site-packages\sklearn\preprocessing\_label.py:878: UserWarning: unknown class(es) ['Achievement', 'Benevolence: caring', 'Benevolence: dependability', 'Conformity: interpersonal', 'Conformity: rules', 'Face', 'Hedonism', 'Humility', 'Power: dominance', 'Power: resources', 'Security: personal', 'Security: societal', 'Self-direction: action', 'Self-direction: thought', 'Stimulation', 'Tradition', 'Universalism: concern', 'Universalism: nature', 'Universalism: objectivity', 'Universalism: tolerance'] will be ignored
  warnings.warn(


ValueError: Found input variables with inconsistent numbers of samples: [5220, 20]

In [ ]:
    # create a list of words in the input text
    input_words = input_text.split()

    # create a list of words in the value category
    category_words = category.split()

    # compute the cosine similarity score between the vectors of the words in the input text and the value category
    input_vector = fits_like_a_glove(input_words)
    category_vector = fits_like_a_glove(category_words)
    input_vector_norm = np.linalg.norm(input_vector)
    category_vector_norm = np.linalg.norm(category_vector)
    if input_vector_norm > 0 and category_vector_norm > 0:
        cosine_similarity = cosine_similarity(input_vector, category_vector)[0][0]
    else:
        cosine_similarity = 0

    return cosine_similarity

# create an empty list to store the cosine similarity scores
cosine_similarities = []

# compute the cosine similarity score for each input text and value category
for input_text in training_data['Cleaned_BOW']:
    cosine_similarities.append([compute_cosine_similarity(input_text, category) for category in all_categories])

# add the cosine similarity scores as a column in the feature matrix
arguments_training['cosine_similarity'] = cosine_similarities

In [30]:
# create a list of all value categories and subcategories
all_categories = []
for category, subcategories in value_categories.items():
    all_categories.append(category)
    for subcategory, _ in subcategories.items():
        all_categories.append(subcategory)

# load the arguments and labels data from the tsv files
arguments_data = {}
with open('arguments-training.tsv') as f:
    for line in f:
        argument_id, conclusion, stance, premise = line.strip().split('\t')
        arguments_data[argument_id] = {
            'premise': premise,
            'conclusion': conclusion,
            'stance': stance,
            'categories': []
        }

with open('labels-training.tsv') as f:
    for line in f:
        argument_id, *labels = line.strip().split('\t')
        arguments_data[argument_id]['categories'] = [
            category
            for category, label in zip(all_categories, labels)
            if label == '1'
        ]

# create a list of all arguments and their corresponding categories and subcategories
arguments = []
categories = []
for argument_id, argument_data in arguments_data.items():
    premise = argument_data['premise']
    conclusion = argument_data['conclusion']
    stance = argument_data['stance']

    # create the argument text by concatenating the premiseand conclusion
argument_text = premise + ' ' + conclusion

In [32]:
# create a binary feature for each value category and subcategory
argument_categories = []
for category in all_categories:
    if category in argument_data['categories']:
        argument_categories.append(1)
    else:
        argument_categories.append(0)

# add the argument text and categories to the list
arguments.append(argument_text)
categories.append(argument_categories)

# create a CountVectorizer to convert the argument text to numeric features
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(arguments)

# combine the numeric features with the binary category features
X_with_categories = np.hstack((X.todense(), np.array(categories)))

# split the data into train, test, and validation sets
# X_train, X_test, y_train, y_test = train_test_split(X_with_categories, categories, test_size=0.2)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

# # print the dimensions of the train, test, and validation sets
# print(f'X_train: {X_train.shape}')
# print(f'X_test: {X_test.shape}')
# print(f'X_val: {X_val.shape}')

In [ ]:
def f1_score(predicted_labels, true_labels, which_label=1):
    """
    F1 score is the harmonic mean of precision and recall
    """
    P = precision(predicted_labels, true_labels, which_label=which_label)
    R = recall(predicted_labels, true_labels, which_label=which_label)
    if P and R:
        return 2*P*R/(P+R)
    else:
        return 0.

def avg_f1_score(predicted_labels: List[int], true_labels: List[int], classes: List[int]):
    """
    Calculate the f1-score for each class and return the average of it

    :return: float
    """
    f1_scores = []
    for c in classes:
        f1_scores.append(f1_score(predicted_labels, true_labels, which_label=c))
    return sum(f1_scores)/len(f1_scores)
    # raise NotImplementedError("Calculate the f1-score for each class and return the average of it")

In [34]:
from sklearn.model_selection import train_test_split

# # drop Argument ID column in labels_training
# labels_training_new = labels_training.drop(labels_training.columns[0], axis=1)

# # drop Argument ID column in training_data
# training_data_new = training_data.drop(training_data.columns[0], axis=1)

# split the input texts and labels into train, val, and test sets
X_train, X_test, y_train, y_test = train_test_split(X_with_categories, categories, test_size=.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.5, random_state=42)

ValueError: With n_samples=1, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [115]:
len(X_train), len(X_val), len(X_test), len(y_train), len(y_val), len(y_test), len(training_data), len(labels_training)

(4176, 522, 522, 4176, 522, 522, 5220, 5220)

In [ ]:
# print the dimensions of the train, test, and validation sets
print(f'X_train: {X_train.shape}')
print(f'X_test: {X_test.shape}')
print(f'X_val: {X_val.shape}')

In [116]:
# Train an SVM classifier on the training data
clf = SVC()
clf.fit(X_train, y_train)

# Test the classifier on the test data
# predictions

ValueError: could not convert string to float: 'We should open up three-child policy to encourage childbirth.'

In [109]:
# import the required modules
import xgboost as xgb
from sklearn.model_selection import train_test_split

# create an XGBoost model
model = xgb.XGBClassifier()

# specify the parameters for the model
params = {'max_depth': 3, 'eta': 0.1, 'num_class': 20}

# define a callback function for early stopping
early_stopping_rounds = 10

# create a DMatrix from the training data
dtrain = xgb.DMatrix(X_train, y_train.to_numpy(), enable_categorical=True)

model = xgb.cv(params, dtrain, num_boost_round=1000, early_stopping_rounds=early_stopping_rounds,
               verbose_eval=True)

# get the number of boosting rounds that resulted in the best validation performance
best_iteration = model.best_iteration

# train a new model using the best number of boosting rounds
final_model = xgb.train(params, dtrain, num_boost_round=best_iteration)           

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:Argument ID: object, Conclusion: object, Stance: object, Premise: object, Cleaned_Premise: object, Cleaned_Conclusion: object, Cleaned_BOW: object

In [75]:
print(X_train.dtypes)

Argument ID                   object
Self-direction: thought        int64
Self-direction: action         int64
Stimulation                    int64
Hedonism                       int64
Achievement                    int64
Power: dominance               int64
Power: resources               int64
Face                           int64
Security: personal             int64
Security: societal             int64
Tradition                      int64
Conformity: rules              int64
Conformity: interpersonal      int64
Humility                       int64
Benevolence: caring            int64
Benevolence: dependability     int64
Universalism: concern          int64
Universalism: nature           int64
Universalism: tolerance        int64
Universalism: objectivity      int64
Conclusion                    object
Stance                        object
Premise                       object
Cleaned_Premise               object
Cleaned_Conclusion            object
Cleaned_BOW                   object
d

In [226]:
# Create a new Empath lexicon
lexicon = Empath()

categories = list(value_categories.keys())
# values = list(value_categories.values())

# Create a dictionary of values for each category
values = {}
for category, value_map in value_categories.items():
    values[category] = list(value_map.keys())

# Create the lexicon
for category, values in values.items():
    lexicon.create_category(category, values)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [ ]:
# load the arguments and labels data from the tsv files
arguments_data = {}
with open('arguments-training.tsv') as f:
    for line in f:
        argument_id, conclusion, stance, premise = line.strip().split('\t')
        arguments_data[argument_id] = {
            'premise': premise,
            'conclusion': conclusion,
            'stance': stance,
            'categories': []
        }

with open('labels-training.tsv') as f:
    for line in f:
        argument_id, *labels = line.strip().split('\t')
        arguments_data[argument_id]['categories'] = [
            category
            for category, label in zip(all_categories, labels)
            if label == '1'
        ]

from sklearn.feature_extraction.text import CountVectorizer

# load the value-categories.json file
with open('value-categories.json') as json_file:
    value_categories = json.load(json_file)

# create a list of all value categories and subcategories
all_categories = []
for category, subcategories in value_categories.items():
    all_categories.append(category)
    for subcategory, _ in subcategories.items():
        all_categories.append(subcategory)

# create a list of all arguments and their corresponding

In [155]:
# Print the categories and their values
for category in lexicon._lexicon:
    print(f"{category}: {lexicon._lexicon[category]}")

AttributeError: 'Empath' object has no attribute '_lexicon'

In [150]:
values.items()

dict_items([('Be logical', ['being better by the numbers but not by gut feeling', 'fostering a rational way of thinking', 'promoting focus and consistency', 'promoting the rational analysis of circumstances', 'promoting the scientific method']), ('Have an objective view', ['fostering to seek the truth', 'fostering to take on a neutral perspective', 'promoting to form an unbiased opinion', 'providing people with the means to make informed decisions', 'weighing all pros and cons'])])

In [132]:
value_categories

{'Self-direction: thought': {'Be creative': ['allowing for more creativity or imagination',
   'being more creative',
   'fostering creativity',
   'promoting imagination'],
  'Be curious': ['being the more interesting option',
   'fostering curiosity',
   'making people more keen to learn',
   'promoting discoveries',
   'sparking interest'],
  'Have freedom of thought': ['allowing people to figure things out on their own',
   'allowing people to make up their mind',
   'resulting in less censorship',
   "resulting in less influence on people's thoughts"]},
 'Self-direction: action': {'Be choosing own goals': ['allowing people to choose what is best for them',
   'allowing people to decide on their life',
   'allowing people to follow their dreams'],
  'Be independent': ['allowing people to plan on their own',
   'resulting in fewer times people have to ask for consent'],
  'Have freedom of action': ['allowing people to be self-determined',
   'allowing people to do things even though

In [116]:
value_categories

['Self-direction: thought',
 'Self-direction: action',
 'Stimulation',
 'Hedonism',
 'Achievement',
 'Power: dominance',
 'Power: resources',
 'Face',
 'Security: personal',
 'Security: societal',
 'Tradition',
 'Conformity: rules',
 'Conformity: interpersonal',
 'Humility',
 'Benevolence: caring',
 'Benevolence: dependability',
 'Universalism: concern',
 'Universalism: nature',
 'Universalism: tolerance',
 'Universalism: objectivity']

In [ ]:
# Function to search for phrases in text
def search_phrases(text, phrases):
    # Iterate through phrases and check if each is present in the text
    for phrase in phrases:
        if phrase in text:
            return True
    return False

# Function to extract features for each argument
def extract_features(text):
    # Initialize dictionary to store features
    features = {}

    # Iterate through the value categories and extract associated words
    for value, words in value_categories.items():
        # Use the search_phrases() function to check if the words are present in the text
        features[value] = search_phrases(text, words)
    
    return features

# Extract features for each argument in the training data
train_features = []
for text in train['Cleaned_BOW']:
    train_features.append(extract_features(text))

# Extract features for each argument in the validation data
val_features = []
for text in val['Cleaned_BOW']:
    val_features.append(extract_features(text))

### Incorporate value-categories in feature extraction:

In [79]:
# Dictionary mapping each human value to a list of associated words
value_dict = {value: words for value, words in value_categories.items()}

# Extract features which check for human value being present in Cleaned BOW
for i in value_dict:
    train[i] = train['Cleaned_BOW'].str.contains("|".join(value_dict[i]), regex=True)

# Concatenate additional features with the word vectors to create a single input vector for each argument
XGB_X = np.concatenate((train_vectors.toarray(), train.drop(['Argument ID', 'Premise', 'Conclusion', 'Stance', 'Cleaned_Premise', 'Cleaned_Conclusion', 'Cleaned_BOW'], axis=1).to_numpy()), axis=1)
XGB_X_val = np.concatenate((val_vectors.toarray(), val.drop(['Argument ID', 'Premise', 'Conclusion', 'Stance', 'Cleaned_Premise', 'Cleaned_Conclusion', 'Cleaned_BOW'], axis=1).to_numpy()), axis=1)

# Train XGB model with the input vectors and value labels as target variable
labels = labels_training.values[:, 1:]
XGB_model = xgboost.XGBClassifier()
XGB_model.fit(XGB_X, labels)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [90]:
XGB_X.shape[1], XGB_X_val.shape[1]

(1020, 1020)

In [81]:
# cheeck F1 and accuracy on the validation set for xgb model
val_preds = XGB_model.predict(XGB_X_val)

In [ ]:
val_labels = val.values[:, 1:]



label_f1s = []
for i in range(val_labels.shape[1]):
    label_f1 = f1_score(val_labels[:, i], val_preds[:, i])
    label_f1s.append(label_f1)

# Calculate average F1-score across all labels
avg_f1 = np.mean(label_f1s)
print("Average F1-score: {:.4f}".format(avg_f1))

# Calculate overall accuracy
acc = (val_preds == val_labels).all(axis=1).mean()
print("Overall accuracy: {:.4f}".format(acc))

In [ ]:
train

In [98]:
# get value counts for each column in train
train['Stimulation'].value_counts()


False    4176
Name: Stimulation, dtype: int64

### SVM Classifier

In [ ]:
# create a OneVsRestClassifier with a linear SVC
clf = OneVsRestClassifier(SVC(kernel='linear', probability=True))

# fit the model to the training data
clf.fit(train_vectors, train['Label'])

### XGB Classifier

In [ ]:
# create an XGBoost classifier
from xgboost import XGBClassifier
xgb = XGBClassifier()

# fit the model to the training data
xgb.fit(train_vectors, train['Label'])

# create a random forest classifier
rf = RandomForestClassifier()

# fit the model to the training data
rf.fit(train_vectors, train['Label'])


### Keras model 1

In [ ]:
# Create a sequential model
model = Sequential()

# Add a dense layer with 128 units and ReLU activation
model.add(Dense(128, activation="relu"))

# Add a dense layer with 20 units (one for each value category) and sigmoid activation
model.add(Dense(20, activation="sigmoid"))

# Compile the model with binary cross-entropy loss and Adam optimizer
model.compile(loss="binary_crossentropy", optimizer="adam")

# Fit the model on the training data
model.fit(X, labels, epochs=10, batch_size=32)

### Keras model 2:

In [ ]:
# Convert the text of the arguments into a sequence of word vectors
X = sequence.pad_sequences(arguments["text"], maxlen=100)

# Create a sequential model
model = Sequential()

# Add an embedding layer with 100-dimensional word vectors and input length of 100
model.add(Embedding(vocabulary_size, 100, input_length=100))

# Add an LSTM layer with 128 units
model.add(LSTM(128))

# Add a dense layer with 20 units (one for each value category) and sigmoid activation
model.add(Dense(20, activation="sigmoid"))

# Compile the model with binary cross-entropy loss and Adam optimizer
model.compile(loss="binary_crossentropy", optimizer="adam")

# Fit the model on the training data
model.fit(X, labels, epochs=10, batch_size=32)